In [1]:
import sys
sys.path.append('/Users/marklidenberg/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat')

from app.imports.runtime import *
from app.scheduler.frontend import *

from app.scheduler.mozzarella import *
from app.scheduler.mozzarella.properties import *

2021-09-05 18:16:07.493 | INFO     | app.db:create_external_db:7 - Creating database session outside of the app.


In [2]:
def _group_intervals(intervals, max_length=None, interval_func=None):
    if not interval_func:
        interval_func = lambda interval: [interval[0], interval[1]]
    groups = []
    
    intervals = list(sorted(intervals, key=lambda interval: interval_func(interval)[0]))

    cur_group = []
    for interval in intervals:
        if not cur_group:
            cur_group.append(interval)
            continue

        if interval_func(cur_group[-1])[-1] == interval_func(interval)[0]:
            # subsequent 
            cur_group.append(interval)
            if max_length and len(cur_group) == max_length:
                groups.append(cur_group)
                cur_group = []
        else:
            # gap between 
            groups.append(cur_group)
            cur_group = [interval]

    if cur_group:
        groups.append(cur_group)

    return groups

def test_group_intervals():
    intervals = [[1,2], [4,5],[2,4], [10,11]]
    intervals = list(sorted(intervals, key=lambda interval: interval[0]))
    assert group_intervals(intervals, max_length=2) == [[[1, 2], [2, 4]], [[4, 5]], [[10, 11]]]

def parse_schedule(wb_obj):
    wb = utils.cast_workbook(wb_obj)
    
    with code('Get merged cells dataframe'):
        ws = wb['Расписание']
        df = pd.DataFrame()
        df['cell'] = ws.merged_cells.ranges

        bound_names = ('x0', 'x1', 'y0', 'y1')

        df['bounds'] = df['cell'].apply(lambda cell: cell.bounds)
        for i in range(4):
            df[bound_names[i]] = df['bounds'].apply(lambda bound: bound[i])

        df['y0'] += 1
        df['y1'] += 1
        df['label'] = df['cell'].apply(lambda cell: cell.start_cell.value)


        df = df.sort_values(by=['x1', 'x0', 'y1', 'y0'])
        
    
    m = BlockMaker("root")

    def parse_block(label, element_label, rows, start_time, length=2):
        with m.row(label, x=start_time, push_func=add_push):
            for i, row_num in enumerate(rows):
                df1 = df[(df['x1'] == row_num) & (df['x0'] >= 4)]
                groups = group_intervals([row for i, row in df1.iterrows()], 
                                         max_length=length, 
                                         interval_func=lambda row: [row['x0'], row['y0']])


                for group in groups:
                    try:
                        boiling_id = int(group[0]['label'].split(' ')[0])
                    except Exception as e:
                        boiling_id = None

                    m.row(element_label, 
                          size=group[-1]['y0'] - group[0]['x0'],
                          x=group[0]['x0'] - 5, # subtract column header
                          boiling_id=boiling_id,
                          line_num=str(i),
                          group=group,
                          label=str(boiling_id),
                          push_func=add_push)

    with code('fetch start times'):
        start_times = []
        for row_num in [1, 24]:
            hour = int(df[(df['x0'] == 5) & (df['x1'] == row_num)].iloc[0]['label'])
            if hour >= 12:
                # yesterday
                hour -= 24
            start_times.append(hour * 12)
    
    # rows for ['1 смена', '2 смена', ...
    split_rows = list(sorted(df[df['y0'] - df['x0'] >= 50]['x1'].unique())) # [2, 23, 32, 39, 60]
    
    parse_block('boilings', 'boiling', [split_rows[0] + i for i in [1, 5, 13, 17]], start_times[0])
    parse_block('cleanings', 'cleaning', [split_rows[0] + 9], start_times[0])

    parse_block('water_meltings', 'melting', [split_rows[1] + 4], start_times[1])

    parse_block('water_packings', 'packing', [split_rows[2] + 1], start_times[1])



    with code('Find rows for salt melting lines'):
        last_melting_row = df[df['label'] == 'посолка']['x1'].max()
        salt_melting_rows = list(range(split_rows[3] + 1, last_melting_row, 4))

    parse_block('salt_meltings', 'melting', salt_melting_rows, start_times[1])

    with code('add salt forming info to packings'):
        df_formings = df[(df['label'] == 'плавление/формирование') & (df['x1'] >= split_rows[3] + 1)]
        with code('fix start times and column header'):
            df_formings['x0'] += start_times[1] - 5
            df_formings['y0'] += start_times[1] - 5
            df_formings['x1'] += start_times[1] - 5
            df_formings['y1'] += start_times[1] - 5

        df_formings = df_formings.sort_values(by='x0')

        for i, row in df_formings.iterrows():
            overlapping = [m for m in m.root['salt_meltings'].children if m.x[0] <= row['x0'] and m.y[0] >= row['x0']]
            melting = delistify([m for m in overlapping if m.x[0] + 6 == row['x0']], single=True) # todo next: make properly, check 
            melting.props.update(melting_end=row['y0'])
    parse_block('salt_packings', 'packing', [split_rows[4] + 1, split_rows[4] + 7], start_times[1])

    return m.root

def parse_boiling_plan(wb_obj):
    wb = utils.cast_workbook(wb_obj)
    df_bp = read_boiling_plan(wb)
    df_bp['line_name'] = df_bp['line'].apply(lambda line: line.name)
    
        
    water_boiling_ids = [b.props['label'] for b in parsed_schedule['water_packings'].children]
    salt_boiling_ids = [b.props['label'] for b in parsed_schedule['salt_packings'].children]
    
    for line_name, grp_line in df_bp.groupby('line_name'):
        if line_name == LineName.WATER:
            boiling_ids = water_boiling_ids
        else:
            boiling_ids = salt_boiling_ids

        for i, (_, grp) in enumerate(grp_line.groupby('group_id')):
            df_bp.loc[grp.index, 'boiling_id'] = boiling_ids[i]
    df_bp['boiling_id'] = df_bp['boiling_id'].astype(int)
    return df_bp

In [3]:
fn = '/Users/marklidenberg/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat/app/data/dynamic/2021-08-11/approved/2021-08-11 Расписание моцарелла.xlsx'
wb = utils.cast_workbook(fn)

In [4]:
def fill_properties(parsed_schedule, df_bp):
    props = MozzarellaProperties()

    # save boiling_model to parsed_schedule blocks
    for block in parsed_schedule.iter(cls=lambda cls: cls in ['boiling', 'melting', 'packing']):
        boiling_group_df = df_bp[df_bp['boiling_id'] == int(block.props['boiling_id'])]
        block.props.update(boiling_group_df=boiling_group_df,
                           line_name=boiling_group_df.iloc[0]['boiling'].line.name,
                           boiling_model=boiling_group_df.iloc[0]['boiling'])
    
    # parse boilings
    boilings = parsed_schedule['boilings']['boiling', True]
    salt_boilings = [b for b in boilings if b.props['line_name'] == LineName.SALT]
    water_boilings = [b for b in boilings if b.props['line_name'] == LineName.WATER]
    
    # filling code is mirroring the pickle parser from app/scheduler/mozzarella/properties.py
    
    props.bar12_present = "1.2" in [sku.form_factor.name for sku in df_bp['sku']]

    with code('2.7, 3.3, 3.6 tanks'):
        _boilings = [b for b in boilings if str(b.props['boiling_model'].percent) == '3.3']
        if _boilings:
            _tank_boilings = [b for i, b in enumerate(_boilings) if (i + 1) % 9 == 0 or i == len(_boilings) - 1]
            props.line33_last_termizator_end_times = [cast_human_time(b.x[0] + (b.props['group'][0]['y0'] - b.props['group'][0]['x0'])) for b in _tank_boilings]

        _boilings = [b for b in boilings if str(b.props['boiling_model'].percent) == '3.6']
        if _boilings:
            _tank_boilings = [b for i, b in enumerate(_boilings) if (i + 1) % 9 == 0 or i == len(_boilings) - 1]
            props.line36_last_termizator_end_times = [cast_human_time(b.x[0] + (b.props['group'][0]['y0'] - b.props['group'][0]['x0'])) for b in _tank_boilings]

        _boilings = [b for b in boilings if str(b.props['boiling_model'].percent) == '2.7']
        if _boilings:
            _tank_boilings = [b for i, b in enumerate(_boilings) if (i + 1) % 9 == 0 or i == len(_boilings) - 1]
            props.line27_last_termizator_end_times = [cast_human_time(b.x[0] + (b.props['group'][0]['y0'] - b.props['group'][0]['x0'])) for b in _tank_boilings]

    with code('multihead'):
        multihead_packings = list(parsed_schedule.iter(cls='packing', boiling_group_df=lambda df: df['sku'].iloc[0].packers[0].name == 'Мультиголова'))
        if multihead_packings:
            props.multihead_end_time = cast_human_time(max(packing.y[0] for packing in multihead_packings))

        water_multihead_packings = list(parsed_schedule.iter(cls='packing', boiling_group_df=lambda df: df['sku'].iloc[0].packers[0].name == 'Мультиголова' and df.iloc[0]['boiling'].line.name == LineName.WATER))
        if water_multihead_packings:
            props.water_multihead_present = True


    with code('cleanings'):
        props.short_cleaning_times = [cast_human_time(cleaning.x[0]) for cleaning in parsed_schedule.iter(cls='cleaning') if 'Короткая мойка' in cleaning.props['group'][0]['label']]
        props.full_cleaning_times = [cast_human_time(cleaning.x[0]) for cleaning in parsed_schedule.iter(cls='cleaning')if 'Полная мойка' in cleaning.props['group'][0]['label']]


    with code('meltings'):
        if salt_boilings:
            props.salt_melting_start_time = cast_human_time(parsed_schedule['salt_meltings']['melting', True][0].x[0] + 6) # todo next: take from parameters


    with code('cheesemakers'):
        values = []
        for b in parsed_schedule['boilings']['boiling', True]:
            values.append([b.props['line_num'], b.y[0]])

        df1 = pd.DataFrame(values, columns=['pouring_line', 'finish'])
        values = df1.groupby('pouring_line').agg(max).to_dict()['finish']
        values = list(sorted(values.items(), key=lambda kv: kv[0]))  # [('0', 116), ('1', 97), ('2', 149), ('3', 160)]
        values_dict = dict(values)
        props.cheesemaker1_end_time = cast_human_time(values_dict.get('0'))
        props.cheesemaker2_end_time = cast_human_time(values_dict.get('1'))
        props.cheesemaker3_end_time = cast_human_time(values_dict.get('2'))
        props.cheesemaker4_end_time = cast_human_time(values_dict.get('3'))


    with code('melting end'):
        def _get_melting_end(line_boilings, water=False):
            if not line_boilings:
                return None
            line_boilings = list(sorted(line_boilings, key=lambda b: b.x[0]))
            last_boiling_id = line_boilings[-1].props['boiling_id']
            last_melting = parsed_schedule.find_one(cls='melting', boiling_id=last_boiling_id)
            if water:
                return last_melting.y[0]
            else:
                return last_melting.props['melting_end']

        props.water_melting_end_time = cast_human_time(_get_melting_end(water_boilings, water=True))
        props.salt_melting_end_time = cast_human_time(_get_melting_end(salt_boilings))


    with code('drenators'):
        with code('fill drenators info'):
            for line_num in range(4):
                line_boilings = [b for b in boilings if b.props['line_num'] == str(line_num)]
                line_boilings = list(sorted(line_boilings, key=lambda b: b.x[0]))
                for i, b in enumerate(line_boilings):
                    b.props.update(drenator_num=i % 2, 
                                   drenator_end=b.y[0] + b.props['boiling_model'].line.chedderization_time // 5 - 5) # todo next: make pouring off properly

        with code('fill drenator properties'):
            # get when drenators end: [[3, 96], [2, 118], [4, 140], [1, 159], [5, 151], [7, 167], [6, 180], [8, 191]]
            values = []
            for boiling in boilings:
                values.append([boiling.props['drenator_end'], boiling.props['line_num'], boiling.props['drenator_num']])
            df = pd.DataFrame(values, columns=['drenator_end', 'pouring_line', 'drenator_num'])
            df['id'] = df['pouring_line'].astype(int) * 2 + df['drenator_num'].astype(int)
            df = df[['id', 'drenator_end']]
            df = df.drop_duplicates(subset='id', keep='last')
            df = df.reset_index(drop=True)
            df['id'] = df['id'].astype(int) + 1

            df = df.sort_values(by='id')

            values = df.values.tolist()
            values_dict = dict(values)
            props.drenator1_end_time = cast_human_time(values_dict.get(1))
            props.drenator2_end_time = cast_human_time(values_dict.get(2))
            props.drenator3_end_time = cast_human_time(values_dict.get(3))
            props.drenator4_end_time = cast_human_time(values_dict.get(4))
            props.drenator5_end_time = cast_human_time(values_dict.get(5))
            props.drenator6_end_time = cast_human_time(values_dict.get(6))
            props.drenator7_end_time = cast_human_time(values_dict.get(7))
            props.drenator8_end_time = cast_human_time(values_dict.get(8))

    
    return props


In [5]:
import yaml
yaml.dump({'a': 1})

'a: 1\n'

In [6]:
parsed_schedule = parse_schedule(fn)
df_bp = parse_boiling_plan(fn)

NameError: name 'group_intervals' is not defined

In [ ]:
parsed_schedule

In [837]:
props = fill_properties(parsed_schedule, df_bp)

In [839]:
dict(props)

{'bar12_present': True,
 'line33_last_termizator_end_times': ['07:40'],
 'line36_last_termizator_end_times': ['04:40'],
 'line27_last_termizator_end_times': ['10:05', '12:05'],
 'multihead_end_time': '18:00',
 'water_multihead_present': True,
 'short_cleaning_times': [],
 'full_cleaning_times': ['13:00', '14:55'],
 'salt_melting_start_time': '07:00',
 'cheesemaker1_end_time': '11:05',
 'cheesemaker2_end_time': '14:20',
 'cheesemaker3_end_time': '16:10',
 'cheesemaker4_end_time': '13:25',
 'water_melting_end_time': '16:15',
 'salt_melting_end_time': '20:05',
 'drenator1_end_time': '14:40',
 'drenator2_end_time': '11:35',
 'drenator3_end_time': '16:55',
 'drenator4_end_time': '13:05',
 'drenator5_end_time': '18:45',
 'drenator6_end_time': '15:05',
 'drenator7_end_time': '14:00',
 'drenator8_end_time': '16:00'}

In [7]:
wb = utils.cast_workbook(fn)

with code('Get merged cells dataframe'):
    ws = wb['Расписание']
    df = pd.DataFrame()
    df['cell'] = ws.merged_cells.ranges

    bound_names = ('x0', 'x1', 'y0', 'y1')

    df['bounds'] = df['cell'].apply(lambda cell: cell.bounds)
    for i in range(4):
        df[bound_names[i]] = df['bounds'].apply(lambda bound: bound[i])

    df['y0'] += 1
    df['y1'] += 1
    df['label'] = df['cell'].apply(lambda cell: cell.start_cell.value)


    df = df.sort_values(by=['x1', 'x0', 'y1', 'y0'])

In [10]:
df[df['label'] == 'Сыроизготовитель №1 Poly 1'].iloc[0]['y0']

5

In [847]:
split_line_rows = list(sorted(df[df['y0'] - df['x0'] >= 50]['y0'].unique()))
split_line_rows

[149, 150, 275, 312]

In [849]:
df[df['y0'] - df['x0'] >= 50]['label'].values.tolist()

['1 смена',
 '2 смена',
 'Оператор + Помощник',
 'бригадир упаковки + 5 рабочих',
 '1 смена оператор + помощник',
 None,
 'Бригадир упаковки +5 рабочих упаковки + наладчик',
 'бригадир + наладчик + 5 рабочих']

In [881]:
df[df['label'] == 'охлаждение']

,cell,bounds,x0,x1,y0,y1,label
1321,BN29:BR29,"(66, 29, 70, 29)",66,29,71,30,охлаждение
1322,BS29:BV29,"(71, 29, 74, 29)",71,29,75,30,охлаждение
1323,CG29:CK29,"(85, 29, 89, 29)",85,29,90,30,охлаждение
1324,CL29:CU29,"(90, 29, 99, 29)",90,29,100,30,охлаждение
1325,CX29:DB29,"(102, 29, 106, 29)",102,29,107,30,охлаждение
1326,DC29:DL29,"(107, 29, 116, 29)",107,29,117,30,охлаждение
1327,DP29:DT29,"(120, 29, 124, 29)",120,29,125,30,охлаждение
1328,DU29:ED29,"(125, 29, 134, 29)",125,29,135,30,охлаждение
1288,EI29:EM29,"(139, 29, 143, 29)",139,29,144,30,охлаждение
1289,EN29:EQ29,"(144, 29, 147, 29)",144,29,148,30,охлаждение
